In [1]:
%%capture --no-stderr
%pip install langchain langgraph langchain-ollama tavily-python

In [2]:
import getpass
import os

# 定义一个帮助函数来检查环境变量，如果不存在则提示用户输入
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"请输入您的 {var}")

# 设置 OpenAI 和 Langchain API 密钥
_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# 可选：在 LangSmith 中添加追踪功能
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [5]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama

writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a knowledgeable Java programming assistant specializing in creating high-quality, clear, and structured content tailored to Java developers."
            " Your task is to provide well-documented code examples, explain concepts clearly, and offer best practices based on the user's request."
            " Focus on producing technical accuracy, readability, and actionable insights to assist in learning or solving Java programming challenges."
            " If the user provides feedback or additional requirements, refine and enhance your response to meet their expectations."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chat_model = ChatOllama(model="llama3.2",temperature="0.5",base_url="http://192.168.22.6:11434", max_tokens=8192, emperature=1.2)

writer = writer_prompt | chat_model


article = ""

topic = HumanMessage(
    content="帮我编写一段java代码，实现高性能排序算法。"
)

for chunk in writer.stream({"messages": [topic]}):
    print(chunk.content, end="")
    article += chunk.content

**Java 高性能排序算法：快速排序**

快速排序（QuickSort）是一种分治性排序算法，具有平均时间复杂度为 O(n log n），在大多数情况下是有效的选择。

### 快速排序算法

```java
public class QuickSort {

    /**
     * 排序算法
     *
     * @param array 传入数组
     */
    public static void sort(int[] array) {
        quickSort(array, 0, array.length - 1);
    }

    /**
     * 快速排序递归方法
     *
     * @param array 传入数组
     * @param low   左边的索引
     * @param high  右边的索引
     */
    private static void quickSort(int[] array, int low, int high) {
        if (low < high) {
            // 选择pivot
            int pivot = partition(array, low, high);

            // 排序左、右部分
            quickSort(array, low, pivot - 1);
            quickSort(array, pivot + 1, high);
        }
    }

    /**
     * 分区方法
     *
     * @param array 传入数组
     * @param low   左边的索引
     * @param high  右边的索引
     * @return 分区后的pivot索引
     */
    private static int partition(int[] array, int low, int high) {
        // 选择中间元素作为pivot
        int pivot = array[high];
        int i = (low - 1);

In [ ]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
       (
            "system",
            "You are a seasoned Java scientist and mentor, reviewing and providing detailed feedback on a programmer's code submission."
            " Your goal is to identify areas for optimization, improve code readability, and ensure adherence to best practices in Java development."
            " Provide actionable recommendations on structure, efficiency, maintainability, and compliance with Java standards."
            " If applicable, suggest refactoring techniques, design patterns, or alternative implementations to enhance the code."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

reflect_chat_model = ChatOllama(model="llama3.2",temperature="0.5",base_url="http://192.168.22.6:11434", max_tokens=8192, emperature=0.2)

reflect = reflection_prompt | reflect_chat_model


reflection = ""

# 将主题（topic）和生成的文章（article）作为输入发送给反思智能体
for chunk in reflect.stream({"messages": [topic, HumanMessage(content=article)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Here are some suggestions for improving the code:

1.  **Consider using a more efficient pivot selection method**: The current implementation uses the last element as the pivot, which can lead to poor performance in cases where the input array is already sorted or nearly sorted. You may want to consider using a more robust pivot selection method, such as the "median of three" approach.
2.  **Use a more efficient sorting algorithm for small arrays**: For small arrays (e.g., fewer than 10 elements), other sorting algorithms like insertion sort or merge sort can be faster and more efficient.
3.  **Improve code readability**: The `partition` method could benefit from additional comments explaining its purpose and the logic behind it.
4.  **Consider using Java 8's Stream API for parallelization**: If you need to sort large datasets, you may want to consider using Java 8's Stream API to take advantage of parallel processing capabilities.
5.  **Add error handling**: Currently, the code assume

In [ ]:
from typing import Annotated  # 用于类型注解
from langgraph.graph import END, StateGraph, START  # 导入状态图的相关常量和类
from langgraph.graph.message import add_messages  # 用于在状态中处理消息
from langgraph.checkpoint.memory import MemorySaver  # 内存保存机制，用于保存检查点
from typing_extensions import TypedDict  # 用于定义带有键值对的字典类型

# 定义状态类，使用TypedDict以保存消息
class State(TypedDict):
    messages: Annotated[list, add_messages]  # 使用注解确保消息列表使用add_messages方法处理

# 异步生成节点函数：生成内容（如作文）
# 输入状态，输出包含新生成消息的状态
async def generation_node(state: State) -> State:
    # 调用生成器(writer)，并将消息存储到新的状态中返回
    return {"messages": [await writer.ainvoke(state['messages'])]}

# 异步反思节点函数：对生成的内容进行反思和反馈
# 输入状态，输出带有反思反馈的状态
async def reflection_node(state: State) -> State:
    # 创建一个消息类型映射，ai消息映射为HumanMessage，human消息映射为AIMessage
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    
    # 处理消息，保持用户的原始请求（第一个消息），转换其余消息的类型
    translated = [state['messages'][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state['messages'][1:]
    ]
    
    # 调用反思器(reflect)，将转换后的消息传入，获取反思结果
    res = await reflect.ainvoke(translated)
    
    # 返回新的状态，其中包含反思后的消息
    return {"messages": [HumanMessage(content=res.content)]}

In [ ]:
MAX_ROUND = 6

# 定义条件函数，决定是否继续反思过程
# 如果消息数量超过6条，则终止流程
def should_continue(state: State):
    if len(state["messages"]) > MAX_ROUND:
        return END  # 达到条件时，流程结束
    return "reflect"  # 否则继续进入反思节点

In [ ]:
# 创建状态图，传入初始状态结构
builder = StateGraph(State)

# 在状态图中添加"writer"节点，节点负责生成内容
builder.add_node("writer", generation_node)

# 在状态图中添加"reflect"节点，节点负责生成反思反馈
builder.add_node("reflect", reflection_node)

# 定义起始状态到"writer"节点的边，从起点开始调用生成器
builder.add_edge(START, "writer")


# 在"writer"节点和"reflect"节点之间添加条件边
# 判断是否需要继续反思，或者结束
builder.add_conditional_edges("writer", should_continue)

# 添加从"reflect"节点回到"writer"节点的边，进行反复的生成-反思循环
builder.add_edge("reflect", "writer")

# 创建内存保存机制，允许在流程中保存中间状态和检查点
memory = MemorySaver()

# 编译状态图，使用检查点机制
graph = builder.compile(checkpointer=memory)

In [ ]:
# 可视化图
from IPython.display import Image, display

try:
    display(
        Image(
            graph.get_graph(xray=True).draw_mermaid_png()
        )
    )
except Exception as e:
    print(f"Error generating graph: {e}")

In [ ]:
from IPython.display import Markdown, display

# 定义装饰器，记录函数调用次数
def track_steps(func):
    step_counter = {'count': 0}  # 用于记录调用次数
    
    def wrapper(event, *args, **kwargs):
        # 增加调用次数
        step_counter['count'] += 1
        # 在函数调用之前打印 step
        display(Markdown(f"## Round {step_counter['count']}"))
        # 调用原始函数
        return func(event, *args, **kwargs)
    
    return wrapper

# 使用装饰器装饰 pretty_print_event_markdown 函数
@track_steps
def pretty_print_event_markdown(event):
    # 如果是生成写作部分
    if 'writer' in event:
        generate_md = "#### 写作生成:\n"
        for message in event['writer']['messages']:
            generate_md += f"- {message.content}\n"
        display(Markdown(generate_md))
    
    # 如果是反思评论部分
    if 'reflect' in event:
        reflect_md = "#### 评论反思:\n"
        for message in event['reflect']['messages']:
            reflect_md += f"- {message.content}\n"
        display(Markdown(reflect_md))

In [ ]:
inputs = {
    "messages": [
        HumanMessage(content="帮我使用java语言编写一个简单的5子棋游戏")
    ],
}

config = {"configurable": {"thread_id": "1"}}

async for event in graph.astream(inputs, config):
    pretty_print_event_markdown(event)